In [1]:

import sys
sys.path.append(r"D:\8th Semester\Machine learning\ML Labs")

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
#from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model

import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [3]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [4]:
def MLP():
    model = Sequential()
    model.add(Flatten(input_shape=(time_steps , num_features)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    return model

In [5]:
model1 = MLP()
model1.summary()

C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 504)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │          16,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,193 (63.25 KB)

 Trainable params: 16,193 (63.25 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [7]:
checkpoints = r'D:\8th Semester\Machine learning\ML Labs\lab7\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'D:\8th Semester\Machine learning\ML Labs\lab7'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [8]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [9]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =MLP()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [10]:
import os
path_dataset =r'D:\8th Semester\Machine learning\ML Labs\DATASET'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [11]:
df_te.head(1)

,aep,Is_holiday1,Is_holiday2,Is_Weekend1,Is_Weekend2,sin_month,cos_month,sin_week,cos_week,sin_hour,...,sin_wintert,cos_wintert,sin_springt,cos_springt,sin_summert,cos_summert,sin_fallt,cos_fallt,sin_year_dayt,cos_year_dayt
0,0.518532,1.0,0.0,1.0,0.0,1.0,6.123234e-17,0.866025,-0.5,-0.707107,...,0.0,1.0,1.0,6.123234e-17,0.0,1.0,0.0,1.0,0.956235,0.2926


In [12]:
test_set[0]

array([ 5.18532385e-01,  1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  6.12323400e-17,  8.66025404e-01,
       -5.00000000e-01, -7.07106781e-01, -7.07106781e-01,  0.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  6.12323400e-17,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  1.00000000e+00,  9.56234827e-01,
        2.92600336e-01])

In [13]:
time_steps=2
num_features=21

In [14]:
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=0)

In [15]:
train_X[0]

array([[ 1.69100212e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         2.58819045e-01,  9.65925826e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.41395233e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         5.00000000e-01,  8.66025404e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02]])

In [16]:
train_y[0]

array([], dtype=float64)

In [17]:
time_steps =24
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=1,target_len=2)

In [18]:
train_X[0]

array([[ 1.69100212e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         2.58819045e-01,  9.65925826e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.41395233e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         5.00000000e-01,  8.66025404e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.26232372e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e

In [19]:
train_y[0]

array([1., 1.])

In [20]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.4415297508239746 sec


In [21]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                    verbose = verbose)

Epoch 1/10
2636/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0678 - mae: 0.0678 - mape: 363.3296
Epoch 1: val_loss improved from inf to 0.02550, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\\E1-cp-0001-loss0.03.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.0676 - mae: 0.0676 - mape: 367.2362 - val_loss: 0.0255 - val_mae: 0.0255 - val_mape: 11.1075
Epoch 2/10
2636/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0276 - mae: 0.0276 - mape: 2438.1782
Epoch 2: val_loss improved from 0.02550 to 0.02298, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\\E1-cp-0002-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0276 - mae: 0.0276 - mape: 2427.1616 - val_loss: 0.0230 - val_mae: 0.0230 - val_mape: 10.7862
Epoch 3/10
2633/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0233 - mae: 0.0233 - mape: 1556.8860
Epoch 3: val_loss improved from 0.02298 to 0.01784, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\\E1-cp-0003-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0233 - mae: 0.0233 - mape: 1550.0063 - val_loss: 0.0178 - val_mae: 0.0178 - val_mape: 7.4833
Epoch 4/10
2632/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0215 - mae: 0.0215 - mape: 594.2582
Epoch 4: val_loss improved from 0.01784 to 0.01749, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\\E1-cp-0004-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0214 - mae: 0.0214 - mape: 594.6924 - val_loss: 0.0175 - val_mae: 0.0175 - val_mape: 7.7503
Epoch 5/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0196 - mae: 0.0196 - mape: 470.5777
Epoch 5: val_loss did not improve from 0.01749
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0196 - mae: 0.0196 - mape: 470.8857 - val_loss: 0.0202 - val_mae: 0.0202 - val_mape: 8.4016
Epoch 6/10
2632/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0188 - mae: 0.0188 - mape: 1868.5134
Epoch 6: val_loss improved from 0.01749 to 0.01546, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\\E1-cp-0006-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.0188 - mae: 0.0188 - mape: 1859.6195 - val_loss: 0.0155 - val_mae: 0.0155 - val_mape: 6.5562
Epoch 7/10
2629/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0176 - mae: 0.0176 - mape: 840.9816 
Epoch 7: val_loss did not improve from 0.01546
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0176 - mae: 0.0176 - mape: 837.0310 - val_loss: 0.0225 - val_mae: 0.0225 - val_mape: 8.4984
Epoch 8/10
2631/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0174 - mae: 0.0174 - mape: 1135.0219
Epoch 8: val_loss improved from 0.01546 to 0.01385, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\\E1-cp-0008-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0174 - mae: 0.0174 - mape: 1130.6842 - val_loss: 0.0139 - val_mae: 0.0139 - val_mape: 5.9640
Epoch 9/10
2629/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0167 - mae: 0.0167 - mape: 234.1402
Epoch 9: val_loss did not improve from 0.01385
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0167 - mae: 0.0167 - mape: 234.5168 - val_loss: 0.0174 - val_mae: 0.0174 - val_mape: 8.6371
Epoch 10/10
2635/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0162 - mae: 0.0162 - mape: 528.9769
Epoch 10: val_loss did not improve from 0.01385
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0162 - mae: 0.0162 - mape: 528.6375 - val_loss: 0.0139 - val_mae: 0.0139 - val_mape: 6.0031


In [28]:

model = load_model(r'D:\8th Semester\Machine learning\ML Labs\lab7\E2-cp-0010-loss0.01.h5',
                   custom_objects={'mae': mean_absolute_error})


y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
Mean Absolute Error (MAE): 112.49
Median Absolute Error (MedAE): 89.69
Mean Squared Error (MSE): 21714.57
Root Mean Squared Error (RMSE): 147.36
Mean Absolute Percentage Error (MAPE): 0.77 %
Median Absolute Percentage Error (MDAPE): 0.62 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


# Fine Tuning

In [23]:
checkpoints = r'D:\8th Semester\Machine learning\ML Labs\lab7\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'D:\8th Semester\Machine learning\ML Labs\lab7\E1-cp-0010-loss0.01.h5'
start_epoch= 20

In [24]:
import os
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, InputLayer
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras.backend as K

# Define your model builder function (replace PC.build)
def build_model(time_steps, num_features, reg=0.0005):
    model = Sequential()
    model.add(InputLayer(input_shape=(time_steps, num_features)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(1))  # Adjust output layer size according to your task
    return model


# Path to save/load your model checkpoint
model_path = r'D:\8th Semester\Machine learning\ML Labs\lab7\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'

# Construct the callback to save only the *best* model based on validation loss
EpochCheckpoint1 = ModelCheckpoint(
    model_path,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

# Construct the set of callbacks
callbacks = [EpochCheckpoint1]

# Variable 'model' for holding your model object
model = None

# Check if checkpoint file exists
if not os.path.exists(model_path):
    print("[INFO] No checkpoint found. Compiling new model...")
    model = build_model(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(learning_rate=1e-3)
    model.compile(loss="mae", optimizer=opt, metrics=["mae", "mape"])
else:
    print(f"[INFO] Loading model from {model_path} ...")
    model = load_model(model_path)

    # Update the learning rate
    print(f"[INFO] Old learning rate: {K.get_value(model.optimizer.lr)}")
    K.set_value(model.optimizer.lr, 1e-4)
    print(f"[INFO] New learning rate: {K.get_value(model.optimizer.lr)}")

# Now you can train your model with callbacks
# Example:
# model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=..., callbacks=callbacks)


[INFO] No checkpoint found. Compiling new model...


C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [25]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0463 - mae: 0.0463 - mape: 299.1190 
Epoch 1: val_loss improved from inf to 0.01630, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\E2-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 34s 12ms/step - loss: 0.0463 - mae: 0.0463 - mape: 299.1167 - val_loss: 0.0163 - val_mae: 0.0163 - val_mape: 7.1603
Epoch 2/10
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0127 - mae: 0.0127 - mape: 199.2830 
Epoch 2: val_loss improved from 0.01630 to 0.00936, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\E2-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 30s 11ms/step - loss: 0.0127 - mae: 0.0127 - mape: 199.1149 - val_loss: 0.0094 - val_mae: 0.0094 - val_mape: 4.2334
Epoch 3/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0098 - mae: 0.0098 - mape: 144.3519 
Epoch 3: val_loss improved from 0.00936 to 0.00932, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\E2-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 30s 11ms/step - loss: 0.0098 - mae: 0.0098 - mape: 144.3350 - val_loss: 0.0093 - val_mae: 0.0093 - val_mape: 4.1559
Epoch 4/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0088 - mae: 0.0088 - mape: 25.4365 
Epoch 4: val_loss improved from 0.00932 to 0.00879, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\E2-cp-0004-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - loss: 0.0088 - mae: 0.0088 - mape: 25.4424 - val_loss: 0.0088 - val_mae: 0.0088 - val_mape: 4.3756
Epoch 5/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0083 - mae: 0.0083 - mape: 11.8987
Epoch 5: val_loss improved from 0.00879 to 0.00843, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\E2-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - loss: 0.0083 - mae: 0.0083 - mape: 11.9060 - val_loss: 0.0084 - val_mae: 0.0084 - val_mape: 3.6198
Epoch 6/10
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0080 - mae: 0.0080 - mape: 10.4267
Epoch 6: val_loss improved from 0.00843 to 0.00724, saving model to D:\8th Semester\Machine learning\ML Labs\lab7\E2-cp-0006-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - loss: 0.0080 - mae: 0.0080 - mape: 10.4598 - val_loss: 0.0072 - val_mae: 0.0072 - val_mape: 3.1702
Epoch 7/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0077 - mae: 0.0077 - mape: 21.7784 
Epoch 7: val_loss did not improve from 0.00724
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - loss: 0.0077 - mae: 0.0077 - mape: 21.7826 - val_loss: 0.0087 - val_mae: 0.0087 - val_mape: 3.8195
Epoch 8/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0074 - mae: 0.0074 - mape: 5.2757 
Epoch 8: val_loss did not improve from 0.00724
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 33s 12ms/step - loss: 0.0074 - mae: 0.0074 - mape: 5.2769 - val_loss: 0.0083 - val_mae: 0.0083 - val_mape: 3.5331
Epoch 9/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0072 - mae: 0.0072 - mape: 277.3591 
Epoch 9: val_loss did not improve from 0.00724
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 36s 13ms/step - loss: 0.0072 - mae: 0.0072 - mape: 277.2065 - val_loss: 0.0075 - val_mae

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 31s 12ms/step - loss: 0.0072 - mae: 0.0072 - mape: 83.9967 - val_loss: 0.0067 - val_mae: 0.0067 - val_mape: 2.9282


In [29]:

model = load_model(r'D:\8th Semester\Machine learning\ML Labs\lab7\E2-cp-0007-loss0.01.h5',
                   custom_objects={'mae': mean_absolute_error})
#model = load_model(r'C:\Users\PMLS\shahid (ML-lab)\lab7\E1-cp-0016-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Mean Absolute Error (MAE): 112.49
Median Absolute Error (MedAE): 89.69
Mean Squared Error (MSE): 21714.57
Root Mean Squared Error (RMSE): 147.36
Mean Absolute Percentage Error (MAPE): 0.77 %
Median Absolute Percentage Error (MDAPE): 0.62 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
